<a href="https://colab.research.google.com/github/7benson/SMS-Spam-Classifier/blob/main/SMS_spam_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --no-deps --force-reinstall kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d uciml/sms-spam-collection-dataset

     |████████████████████████████████| 61kB 9.3MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=9a7123a42dc7b613bea385f2e387e4bd8146d251f69e88df2a5449db97386b3e
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


Saving kaggle.json to kaggle.json
  0% 0.00/211k [00:00<?, ?B/s]
100% 211k/211k [00:00<00:00, 91.4MB/s]


In [2]:
file_name='sms-spam-collection-dataset.zip'
from zipfile import ZipFile
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Extraction Done')

Extraction Done


In [3]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [4]:
import pandas as pd
df=pd.read_csv('./spam.csv',usecols=['v1','v2'], encoding='latin-1')
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
corpus=[]
sentences=df.v2
stemmer=PorterStemmer()
for i in range(len(sentences)):
  sent=re.sub('[^A-Za-z]',' ',sentences[i])
  sent=sent.lower()
  words=nltk.word_tokenize(sent)
  words=[stemmer.stem(word) for word in words if word not in stopwords.words('english')]
  sent=' '.join(words)
  corpus.append(sent)
y=pd.get_dummies(df.v1,drop_first=True)

from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000)
corpus=cv.fit_transform(corpus).toarray()

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(corpus,y,test_size=0.2,random_state=42)

from sklearn.linear_model import LogisticRegression
logistic_regression=LogisticRegression()
logistic_regression.fit(X_train,y_train)
y_pred=logistic_regression.predict(X_test)

from sklearn.metrics import accuracy_score,classification_report,f1_score,confusion_matrix
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(f1_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9775784753363229
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       965
           1       1.00      0.83      0.91       150

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115

0.9090909090909091
[[965   0]
 [ 25 125]]


In [6]:
from sklearn.naive_bayes import GaussianNB
logistic_regression=GaussianNB()
logistic_regression.fit(X_train,y_train)
y_pred=logistic_regression.predict(X_test)

from sklearn.metrics import accuracy_score,classification_report,f1_score,confusion_matrix
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(f1_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8618834080717489
              precision    recall  f1-score   support

           0       0.98      0.86      0.91       965
           1       0.49      0.89      0.64       150

    accuracy                           0.86      1115
   macro avg       0.74      0.88      0.77      1115
weighted avg       0.92      0.86      0.88      1115

0.6350710900473934
[[827 138]
 [ 16 134]]


In [17]:
import tensorflow as tf

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=70, activation='relu'))
ann.add(tf.keras.layers.Dense(units=80, activation='relu'))
ann.add(tf.keras.layers.Dense(units=40, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# ann.fit(X_train, y_train, batch_size = 32, epochs = 100,validation_data=(X_test, y_test))
ann.fit(corpus, y, batch_size = 32, epochs = 5,validation_split=0.2)
# corpus,y

Epoch 1/5
140/140 [==============================] - 1s 4ms/step - loss: 0.2151 - accuracy: 0.9271 - val_loss: 0.0563 - val_accuracy: 0.9839
Epoch 2/5
140/140 [==============================] - 1s 4ms/step - loss: 0.0226 - accuracy: 0.9926 - val_loss: 0.0613 - val_accuracy: 0.9857
Epoch 3/5
140/140 [==============================] - 0s 3ms/step - loss: 0.0048 - accuracy: 0.9984 - val_loss: 0.0810 - val_accuracy: 0.9848
Epoch 4/5
140/140 [==============================] - 0s 3ms/step - loss: 0.0014 - accuracy: 0.9996 - val_loss: 0.0919 - val_accuracy: 0.9848
Epoch 5/5
140/140 [==============================] - 1s 4ms/step - loss: 5.1738e-04 - accuracy: 1.0000 - val_loss: 0.1050 - val_accuracy: 0.9839


In [18]:
ann.save('sms-spam-classifier.h5')

In [19]:
model = tf.keras.models.load_model('sms-spam-classifier.h5')


In [22]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
corpus=[]
sentences=['You’ve been personally selected to take part in our 2021 Annual Visitor Survey! Tell us what you think of Chrome and to say “Thank You” you’ll receive a chance to get an Apple iPhone 12 Pro!']

stemmer=PorterStemmer()
for i in range(len(sentences)):
  sent=re.sub('[^A-Za-z]',' ',sentences[i])
  sent=sent.lower()
  words=nltk.word_tokenize(sent)
  words=[stemmer.stem(word) for word in words if word not in stopwords.words('english')]
  sent=' '.join(words)
  corpus.append(sent)
y=pd.get_dummies(df.v1,drop_first=True)

corpus=cv.fit_transform(corpus).toarray()


In [ ]:
cv

In [23]:
model.predict(corpus)

ValueError: ignored